# Text als Struktur

Bisher haben wir Texte als reine Abfolge von Zeichen betrachtet, also gewissermaßen in Bezug auf ihren Kerngehalt. Einige strukturierte Informationen lassen sich etwa über reguläre Ausdrücke extrahieren, indem man sich Regelmäßigkeiten in den Texten zu Nutze macht. Texte, die in der Praxis etwa im Internet vorliegen, bieten aber weitere Informationen über ihre Struktur, wie etwa Abschnitte oder Überschriften. Diese Strukturinformationen kann man sich ebenfalls zu Nutze machen, um relevante Informationen zu identifizieren.

## Mehr als Text

Um Texten zusätzliche Informationen wie etwa Textformatierungen oder Textstruktur hinzuzufügen, muss man den Text um Zusatzinformationen anreichern. Das ist die Aufgabe von Textformaten wie z.B. *doc*, *odt* oder *html*. [HTML](http://de.wikipedia.org/wiki/Hypertext_Markup_Language) ist dabei die »Sprache des Internet« und vergleichsweise einfach aufgebaut.

Das Grundprinzip ist es, den Text mit Markierungen oder Auszeichnungen (englisch: Markup) zu versehen, die dann in eine bestimmte Formatierung umgewandelt werden. Das IPython Notebook verfügt über eine entsprechende Funktion, um mit HTML-Markierungen versehenen Text formatiert anzuzeigen:

In [3]:
from IPython.core.display import HTML
HTML('Meinst du <b>diese</b> Zeichen?')

Ein solcher Text hat also zwei Darstellungsformen: Einmal als einfacher Text (*plain text*), in dem das Markup sichtbar ist, und einmal als formatierte Ausgabeversion.
Viele Daten liegen im Internet als HTML-Dateien vor. Diese sind für die Sozialwissenschaften häufig relevantes Analysematerial. Dabei müssen in vielen Fällen zunächst die relevanten Daten aus Internetseiten extrahiert werden. So beschreibt die Wikipedia das sogenannte [Screen Scraping](http://de.wikipedia.org/wiki/Screen_Scraping#Funktionsweise):

> Screen Scraping besteht im Wesentlichen aus zwei Schritten:
>
> * Abrufen von Webseiten
> * Extraktion der relevanten Daten


Wenn man sich die Gesamtstruktur dieser HTML-Dokumente ansieht, kann diese als verzweigte Baumstruktur beschrieben werden:

In [1]:
from lxml import html
doc = html.fromstring('''
<html>
  <head>
    <title>Alles &#252;ber Tiere</title>
  </head>
  <body>
    <div id="content">
      <h1>Welches Tier passt zu mir?</h1>
      <p class="teaser">Das k&#246;nnen Sie auf <b>dieser Seite</b> herausfinden.</p>
      <p>Machen Sie unseren <a href="test.html">Test!</a></p>
    </div>
    <div id="nav">
      <ul>
        <li><a href="katzen.html">Katzen</a></li>
        <li><a href="hunde.html">Hunde</a></li>
      </ul
    </div>
  </body>
</html>
''')

Dieses Dokument besteht aus ineinander geschachtelten Markup-Anweisungen. Diese Struktur lässt sich sichtbar machen, wenn man den Text ignoriert und nur die Strukturinformation anzeigt:

In [5]:
for element in doc.iter():
    indent = len(list(element.iterancestors()))
    print '{space}- {tag}'.format(space=u'  ' * indent,
                                  tag=element.tag)

- html
  - head
    - title
  - body
    - div
      - h1
      - p
        - b
      - p
        - a
    - div
      - ul
        - li
          - a
        - li
          - a


Zumeist sind für eine Analyse nicht alle Informationen relevant, oder Teile Daten sollen voneinander unterschieden werden. In diesem Beispiel wäre es etwa sinnvoll, das Navigationsmenü und ähnliches Beiwerk zu ignorieren, und im Text die Überschrift vom Inhalt zu trennen. Aus der Baumstruktur lassen sich solche Informationen gezielt extrahieren. Dazu gibt es zwei Methoden: XPath und CSS.

### XPath

Die Ordnerstruktur einer Festplatte oder die Navigation einer Website lässt sich auch als Baum beschreiben, wie etwa im [Inhaltsverzeichnis](https://www.unilu.ch/inhaltsverzeichnis/) der Uni Luzern. Ein bestimmtes Element in diesem Baum lässt sich über einen Pfad beschreiben:

![Website des Soziologischen Seminars](Bilder/UniLU.png)

Ähnlich funktioniert dies auch mit dem HTML-Dokument. Dabei können aber an einer Adresse mehrere Elemente sitzen, eine Adresse gibt daher immer eine Liste von passenden Elementen zurück:

In [6]:
doc.xpath('/html/body/div/p')

[<Element p at 0x7f2538202db8>, <Element p at 0x7f2538202f18>]

Dabei ist ein »Element« in diesem Sinne ein Objekt, das auch Meta-Informationen wie Attribute enthält. Diese lassen sich auslesen:

In [2]:
links = doc.xpath('/html/body/div/p/a')
first_link = links[0]
first_link.get('href')

'test.html'

Der Textinhalt eines Elements lässt sich über eine spezielle Adresse `text()` auslesen:

In [3]:
doc.xpath('/html/body/div/h1/text()')

['Welches Tier passt zu mir?']

In [8]:
# Aber:
doc.xpath('/html/body/div/p/text()')

[u'Das k\xf6nnen Sie auf ', ' herausfinden.', 'Machen Sie unseren ']

Wenn das Element selbst wiederum Unterelemente enthält, ist das Ergebnis nicht unbedingt das erwartete: Die Textteile »dieser Seite« und »Test« gehören formal nicht zum Text des **p**-Elements, sondern zu den untergeordneten Elementen **b** und **a**.

Um den gesamten Text eines Elements und seiner Unterelemente als zusammenhängende Zeichenkette zu erhalten, gibt es die Hilfsmethode `text_content()`.

In [4]:
[p.text_content() for p in doc.xpath('/html/body/div/p')]

['Das können Sie auf dieser Seite herausfinden.', 'Machen Sie unseren Test!']

Dabei ist es nicht nur umständlich, jedes Mal den kompletten Pfad anzugeben, in vielen Fällen möchte man auch einen Typ von Element finden, ohne ihren genauen Ort in der Hierarchie zu kennen. Dazu gibt es die Kurznotation `//`, die für beliebige übergeordnete Elemente steht.

In [10]:
doc.xpath('//p')

[<Element p at 0x7f2538202db8>, <Element p at 0x7f2538202f18>]

Reale Webseiten enthalten oft eine Menge zusätzlicher Elemente, die für die Textanalyse störend sind, wie z.B. Navigation, Anzeigen, Kopfzeilen und dekorative Elemente. Um gezielt die relevanten Passagen zu finden, muss man die Pfade oft mit zusätzlichen Filterbedingungen einschränken. Typische Kriterien können hier die HTML-Attribute *id* und *class* sein, die oft zur Strukturierung von Webseiten eingesetzt werden. Hier ist XPath sehr vielseitig. Den einleitenden Absatz, im Journalismus »Teaser« genannt, kann man etwa über die entsprechende Klassenangabe identifizieren:

In [11]:
doc.xpath('//p[@class="teaser"]')

[<Element p at 0x7f2538202db8>]

Die Links des Navigationsbereichs findet man über die ID:

In [12]:
doc.xpath('//div[@id="nav"]//a')

[<Element a at 0x7f2538192418>, <Element a at 0x7f2538202e68>]

### CSS

Die Syntax für diese Bedingungen ist zwar sehr flexibel, aber auch recht kompliziert. In den meisten Fällen bietet sich eine einfachere Variante an, die auf die Besonderheiten von HTML zugeschnitten ist: CSS-Selektoren. [CSS](http://de.wikipedia.org/wiki/Cascading_Style_Sheets) ist eigentlich eine Sprache für die Formatierung von Webseiten. Sie enthält aber eine Möglichkeit, Elemente in HTML-Dokumenten auszuwählen, die sich auch in anderen Kontexten einsetzen lässt. Ihre Hauptelemente sind dabei Element-Namen, IDs und Klassen.

In [13]:
# Element
doc.cssselect('a')

[<Element a at 0x7f79e00848e8>,
 <Element a at 0x7f79e0065310>,
 <Element a at 0x7f79e00d5e68>]

In [14]:
# IDs (z.B. <div id="nav">)
doc.cssselect('#nav a')

[<Element a at 0x7f79e0065310>, <Element a at 0x7f79e00d5e68>]

In [15]:
# Klassen (z.B. <p class="teaser">)
doc.cssselect('p.teaser')  # oder nur '.teaser'

[<Element p at 0x7f79e00651b0>]

Bei IDs und Klassen kann man dabei optional einen Elementnamen mit angeben. So steht `p.teaser` für ein `p`-Element mit der Klasse »teaser«, während `.teaser` ein *beliebiges* Element mit dieser Klasse bezeichnet.

Eine ausführliche Dokumentation der unterschiedlichen Selektoren findet sich [hier](http://wiki.selfhtml.org/wiki/CSS/Selektoren). Ein Hilfsmittel, um CSS-Selektoren zu finden, ist [Selector Gadget](http://selectorgadget.com/).

## Ein echtes Beispiel

Nehmen wir als Beispiel eine [Rede der deutschen Bundeskanzlerin](http://www.bundesregierung.de/Content/DE/Rede/2014/09/2014-09-17-merkel-jugend-forscht.html) von der Website der deutschen Bundesregierung. Wir wollen den Titel, das Datum, den Teaser und den Text dieser Seite für weitere Analysen extrahieren.

In [16]:
rede = html.parse('http://www.bundesregierung.de/Content/DE/Rede/2014/09/2014-09-17-merkel-jugend-forscht.html')
root = rede.getroot()

In [17]:
# Titel
root.cssselect('.text h1')[0].text_content()

u'Rede von Bundeskanzlerin Merkel zum Empfang der Preistr\xe4gerinnen und Preistr\xe4ger des 49. Bundeswettbewerbs \u201eJugend forscht\u201c am 17. September 2014'

In [18]:
# Teaser
root.cssselect('.text .abstract')[0].text_content()

'im Bundeskanzleramt\n'

Das Datum ist etwas komplizierter: Diese Kopfangaben auf der Seite bestehen aus Definitionslisten ([dl](http://wiki.selfhtml.org/wiki/HTML/Textstrukturierung/dl)) mit dem folgenden Aufbau:

```html
<dl>
    <dt>Datum:</dt><dd>17. September 2014</dd>
    <dt>Ort:</dt><dd>Berlin</dd>
</dl>
```

In diesem Fall ist nur das Datum in dieser Form angegeben, aber bei anderen Reden ist dies unterschiedlich. In Python lässt sich das Datum so herausfiltern:

In [19]:
# Datum
labels = root.cssselect('.text dl dt')
contents = root.cssselect('.text dl dd')
date = None
for label, content in zip(labels, contents):
    if 'Datum' in label.text_content():
        date = content.text_content()
        break
date

'17. September 2014'

In [20]:
# Text
pars = root.cssselect('.text .abstract ~ p')  # Alle p-Elemente hinter .abstract
text = '\n\n'.join([par.text_content() for par in pars])
text[0:200]

u'Lieber Herr Baszio,\nliebe Frau Wanka,\nmeine Damen und Herren,\naber vor allem: liebe Preistr\xe4gerinnen und Preistr\xe4ger von \u201eJugend forscht\u201c,\n\nich habe Ihnen einen der spannenden Termine im Kalenderjahr '

Auf diese Weise lassen sich nun die relevanten Inhalte einer Seite herausfiltern.

In der Praxis würde man nicht Seite für Seite einzeln aufrufen, sondern auf ähnliche Weise auf einer Übersichtsseite alle Links zu den einzelnen Texten herausfiltern und diese automatisch abarbeiten. Die Datei *bundesscraper.py* enthält ein solches Programm, das alle Reden von der Seite der Bundesregierung ausliest und in einer CSV-Datei speichert.